In [1]:
from warnings import filterwarnings
filterwarnings('ignore')
import importlib
%matplotlib inline

In [2]:
#!git clone https://github.com/ieee8023/covid-chestxray-dataset.git
#!wget https://data.mendeley.com/datasets/rscbjbr9sj/2/files/f12eaf6d-6023-432f-acc9-80c9d7393433/ChestXRay2017.zip
#!wget https://raw.githubusercontent.com/ashvinijangid/covid-19-x-ray/master/loader.py

In [1]:
from train import train

:	[dataset already present]


In [ ]:
importlib.reload(train)